## Chapter 2 - Prepping Data for AI
This notebook contains examples of data preparation strategies for AI, including data cleaning, feature engineering, and handling data sensitivity. Using open-source tools like Pandas, LangChain, and ChromaDB, it explores design patterns for crafting high-quality datasets. It also covers techniques for ensuring data privacy and security, highlighting methods like data masking and synthetic data generation to safeguard sensitive information.

### Listing 2-1: Defining Dataset Constants
Defining **dataset** URLs as constants for reuse throughout the notebook, enabling easy loading of superhero datasets across multiple cells.

In [ ]:
# Base GitHub repository URL
BASE_URL = "https://opensourceai-book.github.io/code/datasets/"

# Dataset file names
INFO_FILE = "superheroes_info.csv"
INFO_CLEAN_FILE = "superheroes_info_cleansed.csv"
POWERS_FILE = "superheroes_powers.csv"
INFO_POWERS_FILE = "superheroes_info_powers.csv"
PLOTS_FILE = "superheroes_story_plots.csv"

# Construct full dataset URLs
SUPERHEROES_INFO_URL = f"{BASE_URL}{INFO_FILE}"
SUPERHEROES_INFO_CLEAN_URL = f"{BASE_URL}{INFO_CLEAN_FILE}"
SUPERHEROES_POWERS_URL = f"{BASE_URL}{POWERS_FILE}"
SUPERHEROES_INFO_POWERS_URL = f"{BASE_URL}{INFO_POWERS_FILE}"
SUPERHEROES_INFO_PLOTS_URL = f"{BASE_URL}{PLOTS_FILE}"

**Note:** If you see errors about unset constants, rerun the above block. This can happen if your Python runtime disconnects in Colab.

### Listing 2-2: Loading and Displaying Dataset Stats
We load our two superhero-related datasets from provided URLs into a Dataframe and displays basic statistics like number of rows, columns, and column names.


In [ ]:
# Import our good friend Pandas, home of the Dataframe and Numpy
import pandas as pd
import numpy as np

# Using the constants defined earlier in the dictionary for reuse
urls = {
    "Superheroes Info Dataset": SUPERHEROES_INFO_URL,
    "Superheroes Powers Dataset": SUPERHEROES_POWERS_URL
}

# Load datasets and display basic stats
for name, url in urls.items():
    df = pd.read_csv(url)
    print(f"{name}:\n  - Rows: {df.shape[0]}\n  - Columns: {df.shape[1]}"
          f"\n  - Column names: {', '.join(df.columns[:5])}...\n")
    print("="*50 + "\n")

### Listing 2-3: Analyzing and Detecting Sparse Fields
Analyzes the dataset by detecting sparse fields, calculating the percentage of missing data for each column.

In [ ]:
# Load the dataset
df = pd.read_csv(SUPERHEROES_INFO_URL)

# Replace placeholder values '-' and '-99' with NaN for better detection of missing data
df.replace('-', np.nan, inplace=True)
df.replace(-99, np.nan, inplace=True)

# Remove the extraneous counter column (assuming it's the first column)
df.drop(df.columns[0], axis=1, inplace=True)

# Sparse fields assessment: Calculate percentage of missing data for all columns
total_entries = len(df)
sparse_fields = {col: (df[col].isnull().sum() / total_entries) * 100 for col in df.columns}

# Output the necessary information, showing all columns, even with 0% missing data
print("\nSparse Fields (Percentage of Missing Data):")
print("{:<20} {:>10}".format("Column", "Missing %"))
print("-" * 30)
for col, percentage in sparse_fields.items():
    print("{:<20} {:>8.2f}%".format(col, percentage))

### Listing 2-4: Lanchain Function for Inferring Race
Defines a function that use Langchain and an open source model on Hugging Face Hub that queries the LLM for a superhero's introduction year using their name and publisher.

In [ ]:
# Install required packages for Hugging Face and LangChain usage
%pip install -q langchain langchain-community langchain-huggingface

# Import os library for environment variable manipulation
import os

# Set your Hugging Face Hub Access token (replace 'your_token_here'
# with your actual token)
os.environ['HUGGINGFACEHUB_API_TOKEN'] = 'Your_Token_Here'

from langchain_core.prompts import ChatPromptTemplate
from langchain_huggingface import HuggingFaceEndpoint
import re

# Initialize the Mistral language model from HuggingFace
llm = HuggingFaceEndpoint(repo_id='mistralai/Mixtral-8x7B-Instruct-v0.1',
                          temperature=0.2)

# Define a prompt template for querying the race via the LLM
prompt = ChatPromptTemplate.from_messages([
    ('system', 'Provide only the superhero race in one word, surrounded by '
               'parentheses (). If you don’t know, respond with "".'),
    ('human', 'What is the race of {hero_name} from {publisher}?')
])

# Define the chatbot chain where the prompt is sent to the language model
chain = prompt | llm

# Function to query the LLM for the race and extract the response within
# parentheses
def get_race_from_llm(hero_name, publisher):
    response = chain.invoke({'hero_name': hero_name, 'publisher': publisher})
    match = re.search(r'\((\w+)\)', response)  # Extract race from parentheses
    return match.group(1) if match else ""

# Example usage
hero_name = "Spider-Man"
publisher = "Marvel Comics"
race = get_race_from_llm(hero_name, publisher)
print(f"Race of {hero_name}: {race}")

### Listing 2-5: Clean and Normalize Dataset
Cleans and normalizes the dataset by removing unnecessary columns, filling missing values, and applying race-based averages (using langchain based program defined in previous listing).

In [ ]:
# Load dataset
df = pd.read_csv(SUPERHEROES_INFO_URL)

# Step 1: Remove unnecessary columns
df.drop(columns=['Unnamed: 0', 'Eye color', 'Hair color', 'Skin color'],
        inplace=True)

# Step 2: Normalize placeholder values
df.replace('-', np.nan, inplace=True)
df.replace(-99, np.nan, inplace=True)

# Step 3: Use LangChain LLM to fill missing race values
for idx, row in df[df['Race'].isna()].iterrows():
    race = get_race_from_llm(row['name'], row['Publisher'])
    if race:
        df.at[idx, 'Race'] = race
        print(f"Filled race for {row['name']}: {race}")
    else:
        print(f"Could not determine race for {row['name']}")

# Step 4: Fill missing height/weight using race averages
race_grouped = df.groupby('Race')[['Height', 'Weight']].mean()

for race in race_grouped.index:
    avg_height = race_grouped.loc[race, 'Height']
    avg_weight = race_grouped.loc[race, 'Weight']
    df.loc[(df['Race'] == race) & (df['Height'].isnull()), 'Height'] = avg_height
    df.loc[(df['Race'] == race) & (df['Weight'].isnull()), 'Weight'] = avg_weight

# Save cleansed data to CSV
df.to_csv('superheroes_info_cleansed.csv', index=False)

# Output sample of cleaned dataset
print(df[['name', 'Race', 'Height', 'Weight']].sample(10))

### Listing 2-6: Calculating Quality with Gini Coefficient
We calculate the **Gini coefficient** for the "Alignment" column to assess imbalance between categories, helping us evaluate potential skew in model predictions.

In [ ]:
# Import our favorite data minipulation libraries
import pandas as pd
import numpy as np

# Load the Superheroes Info dataset
df_info = pd.read_csv(SUPERHEROES_INFO_URL)

# Function to calculate the Gini coefficient for a given distribution of counts
def gini_coefficient(counts):
    sorted_counts = np.sort(counts)  # Sort counts in ascending order
    n = len(counts)  # Number of counts
    cumulative_values = np.cumsum(sorted_counts)  # Cumulative sum of sorted counts
    index = np.arange(1, n + 1)  # Indices for Gini calculation
    # Calculate Gini coefficient using the formula
    gini = (np.sum((2 * index - n - 1) * sorted_counts)) / (
        n * np.sum(sorted_counts)
    )
    return gini

# Count occurrences of each alignment category (good, bad, neutral) after removing '-'
alignment_counts = df_info_filtered['Alignment'].value_counts()

# Calculate Gini coefficient for the Alignment column
gini_score = gini_coefficient(alignment_counts.values)

# Display the counts and Gini coefficient
print("Alignment Counts (excluding blanks):\n", alignment_counts)
print(f"Gini Coefficient for 'Alignment' categories: {gini_score}")


### Listing 2-7: Data Relevance Using EDA
Analyzes average height, weight, and moral alignment proportions by gender, then formats and prints an easy-to-read table for data relevance.

In [ ]:
# Load the dataset
df = pd.read_csv(SUPERHEROES_INFO_CLEAN_URL)

# Filter for missing values in key columns
df = df.dropna(subset=['Gender', 'Alignment', 'Race', 'Height', 'Weight'])

# Analyze imbalance across categorical columns
categories = ['Gender', 'Alignment', 'Race']

gini_results = {}
for category in categories:
    counts = df[category].value_counts()
    gini_results[category] = gini_coefficient(counts.values)

# Analyze imbalance for discretized height and weight
df['Height_bins'] = pd.cut(df['Height'], bins=5)
df['Weight_bins'] = pd.cut(df['Weight'], bins=5)

gini_results['Height'] = gini_coefficient(df['Height_bins'].value_counts().values)
gini_results['Weight'] = gini_coefficient(df['Weight_bins'].value_counts().values)

# Print Gini coefficients for each category
print("Gini Coefficients for Dataset Imbalances:")
for category, gini_score in gini_results.items():
    print(f"{category}: {gini_score:.3f}")

### Listing 2-8: Superhero Dataset Merge Analysis
Analyze compatibility of superheroes_info_clean and superheroes_powers by merging on hero_names field and calculating match percentage for **feature integration depth**.

In [ ]:
# Import pandas
import pandas as pd

# Load the datasets from the URLs
info_df = pd.read_csv(SUPERHEROES_INFO_CLEAN_URL)
powers_df = pd.read_csv(SUPERHEROES_POWERS_URL)

# Rename 'name' in info_df to 'hero_names' for consistent merging
info_df.rename(columns={'name': 'hero_names'}, inplace=True)

# Merge the datasets on 'hero_names'
merged_df = pd.merge(info_df, powers_df, on='hero_names', how='inner')

# Calculate and display the total number of matched entries and the percentage match
matched_count = merged_df.shape[0]
total_info_count = info_df.shape[0]
percentage_matched = (matched_count / total_info_count) * 100

print(f"Matched entries: {matched_count}")
print(f"Total entries in Info dataset: {total_info_count}")
print(f"Percentage matched: {percentage_matched:.2f}%")


### Listing 2-9: Hero Power Metrics Enhancement
Calculates Offensive Power Rating (OPR) and Strategic Defense Rating (SDR) for each hero by summing relevant power attributes, enriching the dataset for enhanced power analysis.

In [ ]:
import pandas as pd

# Load the datasets
info_dfs = pd.read_csv(SUPERHEROES_INFO_CLEAN_URL)
powers_dfs = pd.read_csv(SUPERHEROES_POWERS_URL)

# Ensure consistent naming by renaming 'hero_names' to 'name' in powers_dfs
powers_dfs.rename(columns={'hero_names': 'name'}, inplace=True)

# Define power attributes for OPR and SDR calculations
OFFENSIVE_POWERS = [
    'Super Strength', 'Energy Blasts', 'Weapons Master', 'Telekinesis'
]
DEFENSIVE_POWERS = [
    'Invulnerability', 'Energy Absorption', 'Teleportation', 'Force Fields'
]

# Calculate OPR and SDR in the powers dataset
powers_dfs['OPR'] = powers_dfs[OFFENSIVE_POWERS].sum(axis=1)
powers_dfs['SDR'] = powers_dfs[DEFENSIVE_POWERS].sum(axis=1)

# Keep only necessary columns (name, OPR, SDR)
powers_ratings = powers_dfs[['name', 'OPR', 'SDR']]

# Merge the calculated fields into the info dataset
info_with_ratings = pd.merge(info_dfs, powers_ratings, on='name', how='left')

# Save the updated info dataset with OPR and SDR
info_with_ratings.to_csv(INFO_POWERS_FILE, index=False)
print(f"Updated dataset saved as {INFO_POWERS_FILE} with OPR and SDR.")

### Listing 2-10: Generate Story Plot Dataset

In [ ]:
# Import necessary libraries for building a LangChain chatbot
from langchain_core.prompts import ChatPromptTemplate
from langchain_huggingface import HuggingFaceEndpoint
import pandas as pd
import random  # For random selection

# Configuration
LANGUAGE_MODEL = "mistralai/Mixtral-8x7B-Instruct-v0.1"  # Hugging Face model
TEMP = 0.4  # Temperature for varied responses
NUM_PLOT_SAMPLES = 5  # Number of plot samples to generate for testing

# Initialize the language model
llm = HuggingFaceEndpoint(repo_id=LANGUAGE_MODEL, temperature=TEMP)

# Define a prompt template for generating superhero plots
prompt = ChatPromptTemplate.from_messages([
    ('system', 'Generate a concise superhero plot where:'),
    ('human', '''
    - Hero archetype: {hero}
    - Villain archetype: {villain}
    - Conflict type: {conflict_type}
    - Setting: {setting}
    Include a title and three key plot points.
    ''')
])

# Define the plot generation chain
chain = prompt | llm

# Expanded components for plot generation
heroes = [
    "reluctant hero", "outcast", "chosen one", "antihero", "reluctant mentor",
    "alien protector", "cyber-enhanced rebel", "fallen warrior", "mystic sage",
    "time-traveling guardian", "reformed villain", "empathic healer",
    "street-level hero", "artificial intelligence (AI)", "mystical guardian",
    "scientific genius", "nature warrior", "cosmic nomad", "avenger of loss",
    "dimension traveler", "energy manipulator"
]

villains = [
    "mastermind", "alien invader", "corrupt hero", "rogue AI", "dark sorcerer",
    "doppelgänger", "genetic experiment", "shadow manipulator", "void entity",
    "cyber overlord", "time manipulator", "mythical creature", "corrupt politician",
    "revenge-seeking nemesis", "intergalactic tyrant", "poisonous rival",
    "mind controller", "techno-terrorist", "corrupted scientist", "necromancer",
    "cosmic parasite"
]

conflicts = [
    "personal vendetta", "cosmic invasion", "technological catastrophe", "race against time",
    "internal struggle", "resource battle", "power corruption", "revenge plot",
    "city under siege", "reality manipulation", "identity crisis", "magic vs. science",
    "clash of ideals", "control of the mind", "forbidden knowledge", "mutant uprising",
    "alien invasion", "technological sabotage", "environmental disaster",
    "lost relic chase", "temporal war"
]

settings = [
    "futuristic city", "ancient ruins", "space station", "post-apocalyptic wasteland",
    "underwater fortress", "cyberpunk metropolis", "forest of lost souls",
    "deserted asylum", "alien planet", "hidden temple", "high-tech lab",
    "floating island", "deep jungle", "mystic caves", "parallel universe",
    "underground society", "cloud city", "dreamscape", "hollow mountain",
    "time-frozen world", "ruins of civilization"
]

# Generate a set number of plot samples for testing with random selection
plot_samples = []
for i in range(NUM_PLOT_SAMPLES):
    # Randomly select a hero, villain, conflict, and setting for each plot
    hero = random.choice(heroes)
    villain = random.choice(villains)
    conflict = random.choice(conflicts)
    setting = random.choice(settings)

    # Generate a plot with the chosen elements
    response = chain.invoke({
        'hero': hero,
        'villain': villain,
        'conflict_type': conflict,
        'setting': setting
    })

    # Replace newline characters in the plot response with a separator (|) for CSV compatibility
    response_cleaned = response.replace("\n", " | ")

    # Store the generated plot in a dictionary
    plot_samples.append({
        "Hero": hero,
        "Villain": villain,
        "Conflict Type": conflict,
        "Setting": setting,
        "Plot": response_cleaned
    })

    # Print the generated plot
    print(f"\nPlot {i + 1}:")
    print(f"Hero: {hero}")
    print(f"Villain: {villain}")
    print(f"Conflict Type: {conflict}")
    print(f"Setting: {setting}")
    print(f"Plot:\n{response}")

# Save the plot samples to a CSV
df = pd.DataFrame(plot_samples)
df.to_csv(PLOTS_FILE, index=False)
print(f"\nPlot samples saved to {PLOTS_FILE}.")

### Listing 2-11 Comic Story Assistant with RAG
Showcases a Retrieval-Augmented Generation (RAG) approach that uses ChromaDB with hero plot and power data to dynamically generate tailored superhero story arcs.

In [ ]:
%pip install -q langchain langchain-community langchain-huggingface langchain-openai chromadb

In [ ]:
import os
# Set OpenAI key for accessing GPT-3 and GPT-3 models (paid account)
os.environ["OPENAI_API_KEY"] = "Your_Token_Here"

In [ ]:
# Import necessary libraries
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain.vectorstores.chroma import Chroma
import pandas as pd
import random

# Load datasets from URLs
plot_data = pd.read_csv(SUPERHEROES_INFO_PLOTS_URL)
hero_data = pd.read_csv(SUPERHEROES_INFO_POWERS_URL)

# Prepare plot data for embeddings
plot_texts = plot_data['Plot'].tolist()

# Initialize embeddings model and create Chroma vector database for plot data
embeddings = OpenAIEmbeddings()
db = Chroma.from_texts(plot_texts, embeddings)

# Define query from the comic writer
query_text = ("Show me a story where a time-traveling hero faces a scientific "
              "mastermind in a futuristic city.")

# Retrieve the most relevant plot based on similarity
results = db.similarity_search(query_text, 1)
selected_plot = results[0].page_content

# Randomly select hero and villain from hero data (filtered by alignment)
hero_row = hero_data[hero_data['Alignment'] == 'good'].sample(1).iloc[0]
villain_row = hero_data[hero_data['Alignment'] == 'bad'].sample(1).iloc[0]

# Extract hero and villain details for prompt
hero_name, hero_opr, hero_sdr = hero_row['name'], hero_row['OPR'], hero_row['SDR']
villain_name, villain_opr, villain_sdr = villain_row['name'], villain_row['OPR'], villain_row['SDR']

# Define prompt template for story creation
prompt = ChatPromptTemplate.from_messages([
    ("system", "Generate a superhero story based on the following details:"),
    ("human", '''
    Plot Outline: {plot}

    Hero: {hero} with an offense power of {hero_opr}x and defense of {hero_sdr}x.
    Villain: {villain} with an offense power of {villain_opr}x and defense
    of {villain_sdr}x.

    Rewrite the story with {hero} as the main hero and {villain} as the main villain.
    Build tension, introduce challenges, and create a climactic final showdown.
    ''')
])

print(f"""Hero: {hero_name} with OPR of {hero_opr} and defense of {hero_sdr}x.
Villain: {villain_name} with OPR of {villain_opr}x and defense of {villain_sdr}x.
""")

# Set up LangChain LLM for generating story arcs
llm = ChatOpenAI(temperature=0.2)
chain = prompt | llm

# Invoke the LLM with structured prompt
response = chain.invoke({
    "plot": selected_plot,
    "hero": hero_name,
    "villain": villain_name,
    "hero_opr": hero_opr,
    "hero_sdr": hero_sdr,
    "villain_opr": villain_opr,
    "villain_sdr": villain_sdr
})

# Display the generated story arc
print("Generated Story Arc:")
print(response.content)



### Listing 2-12: Pseudonymizing Superhero Plots Using SpaCy
Demonstrates pseudonymization of entities within superhero plots, using spaCy to replace names, organizations, and locations with generic terms for privacy. **Note:** Be sure to run the *pip* install before running the code snippet below.

In [ ]:
%pip install -q spacy

In [ ]:
import spacy
import pandas as pd

# Load spaCy's English model for entity recognition
nlp = spacy.load('en_core_web_sm')

# Example dataset with superhero story plots
plot_data = pd.read_csv(SUPERHEROES_INFO_PLOTS_URL)

# Function to pseudonymize entity names in the plot text
def pseudonymize_entities(text):
    doc = nlp(text)
    for ent in doc.ents:
        # Replace detected entities with generic labels
        if ent.label_ == "PERSON":
            text = text.replace(ent.text, 'Hero A')
        elif ent.label_ == "ORG":
            text = text.replace(ent.text, 'Organization X')
        elif ent.label_ == "GPE":
            text = text.replace(ent.text, 'Location Z')
    return text

# Pseudonymize only the first plot
first_plot = plot_data['Plot'].iloc[0]
pseudonymized_first_plot = pseudonymize_entities(first_plot)

# Display the pseudonymized first plot
print("Pseudonymized Plot:")
print(pseudonymized_first_plot)

### Listing 2-13: Data Masking And Differential Privacy
Demonstrates data masking and differential privacy on health records by masking phone numbers and adding noise to age values.

In [ ]:
import pandas as pd
import numpy as np

# Sample dataset: health records
data = pd.DataFrame({
    'patient_id': ['A123', 'B456', 'C789'],
    'phone': ['123-456-7890', '987-654-3210', '555-123-4567'],
    'age': [29, 47, 35],
    'diagnosis': ['Condition A', 'Condition B', 'Condition A']
})

# Data Masking: Mask all but the last four digits of phone numbers
data['masked_phone'] = data['phone'].apply(lambda x: 'XXX-XXX-' + x[-4:])

# Differential Privacy: Add noise to age values for anonymization
noise_level = 2  # Adjust noise level as needed
data['age_noisy'] = data['age'] + np.random.laplace(0, noise_level, len(data))

# Display the modified dataset
print("Anonymized Data:\n", data[['patient_id', 'masked_phone',
                                 'age_noisy', 'diagnosis']])

### Listing 2-14: Encrypting Sensitive Data
This code encrypts a sensitive dataset with **Fernet**, allowing secure decryption and access for authorized users only.
**Note:** Be sure to run the following *pip install*

In [ ]:
%pip -q install cryptography

In [ ]:
#Import Fernet
from cryptography.fernet import Fernet
import pandas as pd
import io  # Import io for StringIO

# Generate encryption key and create a cipher suite
key = Fernet.generate_key()
cipher_suite = Fernet(key)

# Sample sensitive data to be encrypted
data = pd.DataFrame({"Patient": ["John Doe", "Jane Smith"],
                     "Diagnosis": ["Diabetes", "Hypertension"]})
data_str = data.to_csv(index=False)
encrypted_data = cipher_suite.encrypt(data_str.encode())

# Show part of the encrypted string
print("Encrypted Data (partial):", encrypted_data[:50], "...")

# Decrypt the data when access is needed
decrypted_data_str = cipher_suite.decrypt(encrypted_data).decode()
secure_data = pd.read_csv(io.StringIO(decrypted_data_str))

print("\nDecrypted Data Accessible (Only by Authorized Users)\n", secure_data)

### Listing 2-15: Generating Synthetic Health Records
Uses **Faker** to create fictional health records, providing a safe, realistic dataset structure for AI training or testing applications. **Note**: Be sure to run the following pip install.

In [ ]:
%pip -q install Faker

In [ ]:
from faker import Faker
import pandas as pd

# Initialize Faker for synthetic data generation
fake = Faker()

# Generate a synthetic health records dataset
data = pd.DataFrame({
    'Patient_ID': [fake.uuid4() for _ in range(5)],
    'Name': [fake.name() for _ in range(5)],
    'Age': [fake.random_int(min=18, max=90) for _ in range(5)],
    'Diagnosis': [fake.random_element(elements=('Condition A',
                                               'Condition B',
                                               'Condition C'))
                   for _ in range(5)],
    'Phone': [fake.phone_number() for _ in range(5)],
    'Address': [fake.address().replace("\n", ", ") for _ in range(5)],
    'Last_Visit': [fake.date_between(start_date='-2y', end_date='today')
                   for _ in range(5)]
})

# Display the synthetic dataset
print(data)

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain import HuggingFaceHub
import re

# Initialize the Mistral language model from HuggingFace
llm = HuggingFaceHub(repo_id='mistralai/Mistral-Small-Instruct-2409', model_kwargs={'temperature': 0.1})

# Define a prompt template for querying the introduction year via the LLM
prompt = ChatPromptTemplate.from_messages([
    ('system', 'Provide only the superhero introduction year in 4 characters. If you don’t know, respond with "".'),
    ('human', 'When was {hero_name} introduced by {publisher}?')
])

# Define the chatbot chain where the prompt is sent to the language model
chain = prompt | llm

# Function to query the LLM for the introduction year and extract just the year
def get_intro_year_from_llm(hero_name, publisher):
    # Invoke the chatbot chain with hero name and publisher
    response = chain.invoke({'hero_name': hero_name, 'publisher': publisher})
    # Use a regex to extract the year from the response (e.g., 1963 or 2015)
    match = re.search(r'\b\d{4}\b', response)
    return match.group(0) if match else ""

### Block - 2: Enrich Superhero Dataset with Years
This block loads a dataset, uses the function to get superhero intro years, updates the dataset, and saves it as a new CSV file.

In [ ]:
# Load the superheroes dataset from the provided URL
df_info = pd.read_csv(SUPERHEROES_INFO_URL)

# Option to process the first 10 rows or all rows
# df_info_sample = df_info.copy()  # Uncomment the line below to process all rows
df_info_sample = df_info.head(10).copy()  # Process only the first 10 rows

# Iterate through each row, passing both name and publisher for the search
for idx in df_info_sample.index:
    hero_name = df_info_sample.loc[idx, 'name']  # Extract superhero's name
    publisher = df_info_sample.loc[idx, 'Publisher']  # Extract publisher's name
    # Use LLM for intro year and update the dataframe
    intro_year = get_intro_year_from_llm(hero_name, publisher)
    df_info_sample.loc[idx, 'Intro Year'] = intro_year  # Safely update the dataframe
    print(f"Processed {hero_name}, Intro Year: {intro_year}")  # Display progress

    # Delay between requests to avoid overwhelming servers (1-second delay)
    time.sleep(1)

# Save the updated dataframe to a new CSV file
# df_info_sample.to_csv('superheroes_info_with_intro_year_sample.csv', index=False)
# print("Dataset updated and saved to 'superheroes_info_with_intro_year_sample.csv'.")